In [1]:
# Q1
import torch
import torch.nn.functional as F
image = torch.rand(6,6)
print("image=", image)
#Add a new dimension along 0th dimension
#i.e. (6,6) becomes (1,6,6). This is because
#pytorch expects the input to conv2D as 4d tensor
image = image.unsqueeze(dim=0)
print("image.shape=", image.shape)
image = image.unsqueeze(dim=0)
print("image.shape=", image.shape)
print("image=", image)
kernel = torch.ones(3,3)
#kernel = torch.rand(3,3)
print("kernel=", kernel)
kernel = kernel.unsqueeze(dim=0)
kernel = kernel.unsqueeze(dim=0)
#Perform the convolution
outimage = F.conv2d(image, kernel, stride=1, padding=0)
print("outimage=", outimage)

image= tensor([[0.3644, 0.3050, 0.9278, 0.1052, 0.6828, 0.5182],
        [0.1996, 0.1654, 0.3114, 0.0051, 0.5672, 0.7513],
        [0.7540, 0.1651, 0.5668, 0.6994, 0.7762, 0.3250],
        [0.8696, 0.7020, 0.4507, 0.2992, 0.8078, 0.2905],
        [0.2020, 0.1711, 0.2923, 0.2807, 0.0608, 0.0678],
        [0.3181, 0.0740, 0.4829, 0.9101, 0.3511, 0.3971]])
image.shape= torch.Size([1, 6, 6])
image.shape= torch.Size([1, 1, 6, 6])
image= tensor([[[[0.3644, 0.3050, 0.9278, 0.1052, 0.6828, 0.5182],
          [0.1996, 0.1654, 0.3114, 0.0051, 0.5672, 0.7513],
          [0.7540, 0.1651, 0.5668, 0.6994, 0.7762, 0.3250],
          [0.8696, 0.7020, 0.4507, 0.2992, 0.8078, 0.2905],
          [0.2020, 0.1711, 0.2923, 0.2807, 0.0608, 0.0678],
          [0.3181, 0.0740, 0.4829, 0.9101, 0.3511, 0.3971]]]])
kernel= tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
outimage= tensor([[[[3.7596, 3.2514, 4.6421, 4.4306],
          [4.1846, 3.3652, 4.4840, 4.5219],
          [4.1735, 3.6273, 4

In [2]:
# Q2
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the input image
image = torch.rand(1, 1, 6, 6)  # Shape: (batch_size, channels, height, width)

# Define the Conv2d layer
conv_layer = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3)

# Apply the Conv2d layer to the input image
output = conv_layer(image)

print("Output shape with torch.nn.Conv2d:", output.shape)
# Define the input image
image = torch.rand(1, 1, 6, 6)  # Shape: (batch_size, channels, height, width)

# Define the kernel
kernel = torch.ones(3, 1, 3, 3)  # Shape: (out_channels, in_channels, kernel_height, kernel_width)

# Apply the conv2d operation using torch.nn.functional.conv2d
output = F.conv2d(image, kernel, stride=1, padding=0)

print("Output shape with torch.nn.functional.conv2d:", output.shape)



Output shape with torch.nn.Conv2d: torch.Size([1, 3, 4, 4])
Output shape with torch.nn.functional.conv2d: torch.Size([1, 3, 4, 4])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Define CNN Classifier
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.classification_head = nn.Sequential(
            nn.Linear(128 * 5 * 5, 20),  # Adjusted input size based on the output of the convolutional layers
            nn.ReLU(),
            nn.Linear(20, 10)
        )

    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(x.size(0), -1))

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# Evaluating the model
model.eval()
correct = 0
total = 0
predictions = []
targets = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.numpy())
        targets.extend(labels.numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# Display confusion matrix
conf_matrix = confusion_matrix(targets, predictions)
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, np.arange(10))
plt.yticks(tick_marks, np.arange(10))
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# Print number of learnable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total number of learnable parameters: %d' % total_params)


[1,   100] loss: 0.953
[1,   200] loss: 0.218
[1,   300] loss: 0.146
[1,   400] loss: 0.120
[1,   500] loss: 0.103
[1,   600] loss: 0.097


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Define CNN Classifier with reduced parameters
class ReducedCNNClassifier(nn.Module):
    def __init__(self):
        super(ReducedCNNClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.classification_head = nn.Sequential(
            nn.Linear(64 * 5 * 5, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(x.size(0), -1))

# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    targets = []
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.numpy())
            targets.extend(labels.numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('Accuracy of the network on the test images: %.2f %%' % accuracy)
    return accuracy

# Function to plot percentage drop in parameters vs accuracy
def plot_drop_vs_accuracy(drop_percentage, accuracies):
    plt.plot(drop_percentage, accuracies, marker='o', linestyle='-')
    plt.title('Drop in Parameters vs Accuracy')
    plt.xlabel('Drop in Parameters (%)')
    plt.ylabel('Accuracy (%)')
    plt.grid(True)
    plt.show()

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define parameter reduction configurations
parameter_reduction_configs = [
    {'conv1_channels': 32, 'conv2_channels': 64, 'fc1_out_features': 64},
    {'conv1_channels': 16, 'conv2_channels': 32, 'fc1_out_features': 32},
    {'conv1_channels': 8, 'conv2_channels': 16, 'fc1_out_features': 16}
]

# Train and evaluate models with different parameter reduction configurations
accuracies = []
for config in parameter_reduction_configs:
    model = ReducedCNNClassifier()
    model.net[0] = nn.Conv2d(1, config['conv1_channels'], kernel_size=3)
    model.net[3] = nn.Conv2d(config['conv1_channels'], config['conv2_channels'], kernel_size=3)
    model.classification_head[0] = nn.Linear(config['conv2_channels'] * 5 * 5, config['fc1_out_features'])
    total_params_before = sum(p.numel() for p in model.parameters() if p.requires_grad)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_model(model, trainloader, criterion, optimizer)
    accuracy = evaluate_model(model, testloader)
    total_params_after = sum(p.numel() for p in model.parameters() if p.requires_grad)
    drop_percentage = 100 * (total_params_before - total_params_after) / total_params_before
    print(f'Drop in parameters: {drop_percentage:.2f}%')
    accuracies.append(accuracy)

# Plot drop in parameters vs accuracy
drop_percentage = [100 * (1 - total_params_after / total_params_before) for _ in range(len(parameter_reduction_configs))]
plot_drop_vs_accuracy(drop_percentage, accuracies)
